# Business Proposal

Business problem: What are the best 5 zip codes to invest in?

Business - A real estate investment firm looking to acquire new property to invest in.

Focus - Investing in real estate for high growth zip codes.

Narrowing down zip codes - Look to see where everyone is trying to move to. Find the highest historic ROI from the past 10 years.

Chosen State: Idaho
 - Based off of Census and other surveys deciding to go with Idaho.

Ways to narrow down zipcodes
 - set certain price limits
 - look at cities
     - maybe look at what cities people are moving to
 - look at states
     - maybe look at what states people are moving to
 - look at ROI per county/city/state
 - get rid of any entries that aren't part of a metro
 - get rid of entries that have less than 10 years of data

# Loading all necessary packages and loading data file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [ ]:
data = pd.read_csv("Data/zillow_data.csv")

# EDA

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

## General info about the dataset

In [ ]:
print('The data has {} number of entries. The number of unique RegionNames is {}.'.format(data.shape[0],data['RegionName'].nunique()))
print('So there seems to be no duplicate entries.')

In [ ]:
print('This dataset looks at {} different "states" (includes DC).'.format(data['State'].nunique()))

In [ ]:
print('This dataset looks at median prices of houses of a given zipcode from 1996-04 to 2018-04.')

## Looking at missing values in the dataframe

In [ ]:
#Seeing how many na values are in the dataset
print('There are {} Nan values in the dataset.'.format(data.isna().sum().sum()))

In [ ]:
#Checking to see what columns have na values.
column_names = data.columns
na_columns = []
for name in column_names:
    NumberOfNa = 0
    NumberOfNa = data[name].isna().sum()
    if NumberOfNa > 0:
        na_columns.append(name)
print('There are {} columns that have na values.'.format(len(na_columns)))
print('Only identifier column with na is "Metro", the others are missing values for dates')

In [ ]:
#Looking at the entries with Nan values for Metro
len(data.loc[lambda x: x.Metro.isna()])
data.loc[lambda x: x.Metro.isna()].head()

In [ ]:
# We can drop all the identifiers besides State, City and RegionName (Zipcode)
data_ZipcodeID = data.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)

In [ ]:
# Creating a Dataframe with all entries that have any NA values
null_data = data_ZipcodeID[data_ZipcodeID.isnull().any(axis=1)]
print(len(null_data))
null_data.head()

A lot of the Nan values seem to be coming from possible newer properties that have been built.

In [ ]:
print('The newest zipcode property, aka the last timestamp that has any nan value is {}.'.format(
    null_data.columns[null_data.isnull().any()][-1]))

## Lets narrow the dataset to just Idaho before continuing on EDA

In [ ]:
# Creating Idaho dataframe
Idaho = data_ZipcodeID[data_ZipcodeID['State'] == 'ID']
Idaho.head()

In [ ]:
# Creating a Dataframe with all entries that have any NA values for Idaho
Idaho_null = Idaho[Idaho.isnull().any(axis=1)]
print('There are {} zip codes on record for Idaho.'.format(len(Idaho)))
print('There are {} zip codes that have missing entries'.format(len(Idaho_null)))
Idaho_null.head()

In [ ]:
print('The newest Idaho zipcode property, aka the last timestamp that has any nan value is {}.'.format(
    Idaho_null.columns[Idaho_null.isnull().any()][-1]))

In [ ]:
#Dropping any zipcodes that do not have 10 years worth of data
Idaho = Idaho.drop(Idaho[Idaho['2008-04'].isnull()].index)

#Dropping columns for any previous data older than 10 years.
Idaho.dropna(axis=1, how='any', inplace=True)

print('2 of the Idaho zipcodes did not meet the 10 years worth of data requirement.')
print('The new number of zipcodes of the dataframe is {}.'.format(len(Idaho)))
print('The Idaho dataset now has no nan values and spans from {} to {}'.format(Idaho.columns[3],Idaho.columns[-1]))

In [ ]:
Idaho.head()

## Looking at last 12 months price distribution for Idaho

In [ ]:
Idaho['Last12MonthAvg'] = Idaho.iloc[:,-12:].mean(axis=1)

In [ ]:
Idaho['Last12MonthAvg'].describe()

In [ ]:
plt.figure(figsize=(15,6))
plt.hist(Idaho['Last12MonthAvg'], bins='auto');
plt.ylabel('Number of Zipcodes');
plt.xlabel('Price');

## Calculate ROI (1yr, 5yr, 10yr) and CV

In [ ]:
#Calculate ROI for 1 year, 5 year, 10 year
Idaho['1yr_ROI'] = (Idaho['2018-04'] / Idaho['2017-04']) - 1
Idaho['5yr_ROI'] = (Idaho['2018-04'] / Idaho['2013-04']) - 1
Idaho['10yr_ROI'] = (Idaho['2018-04'] / Idaho['2008-04']) - 1

In [ ]:
#Calculate CV
Idaho['std'] = Idaho.loc[:,'2006-01':'2018-04'].std(axis=1)
Idaho['mean'] = Idaho.loc[:,'2006-01':'2018-04'].mean(axis=1)
Idaho['CV'] = Idaho['std'] / Idaho['mean']

In [ ]:
Idaho

In [ ]:
print('Choosing to use 5yr ROI for historic ROI metric.')
print('The least you should hold a Property or REIT is 5 years to see gains on income/appreciation.')

## Narrowing zipcodes down to historic best 20? 25? Maybe look at a hist to see a divide?

In [ ]:
Idaho['5yr_ROI'].hist()

## Trying to make maps

In [ ]:
import plotly.express as px
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/id_idaho_zip_codes_geo.min.json') as response:
    Idaho_Map = json.load(response)

Idaho_Map["features"][0]

In [ ]:
#Don't run this will make the map but it will be too large of a size, deal with this later.

#fig = px.choropleth(Idaho, geojson=Idaho_Map, color="5yr_ROI",
#                    locations="RegionName", featureidkey="properties.ZCTA5CE10",
#                    projection="mercator", hover_data=['City']
#                   )
#fig.update_geos(fitbounds="locations", visible=True)
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()

## Stopped here

In [ ]:
def ROI_Column(df):
    Number_of_rows=df.shape[0]
    ROI_Col = []

    for row in list(range(Number_of_rows)):
        i = 2
        
        #To find the oldest entry
        while pd.isna(df.iloc[row][i]):
            i+=1
        
        #Calculating the ROI
        ROI_Col.append((df.iloc[row][-1] - df.iloc[row][i])/df.iloc[row][i])
        
    #Adding the new ROI Column        
    df['ROI'] = ROI_Col
    
    return df

In [ ]:
test2 = ROI_Column(data_ZipcodeID)
test2.head()

In [ ]:
#Maybe not best to do ROI from way back when, dont want to be holding property for 22 years.
#Maybe do ROI from the latest 10 years?

## Looking at latest price distribution of each zipcode

In [ ]:
data['2018-04'].describe()

In [ ]:
data['2018-04'].min()

In [ ]:
# Shows the hist plot of zipcodes with prices between 50,000-3,000,000
plt.hist(np.clip(data['2018-04'], 50000, 3000000), bins='auto');

## Playground

To do:
- look at trends
- maybe look in rolling mean
- zoom in annomalies?

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [ ]:
def melt_data(df):
    
    #Makes the individual time columns into one column and each different time into individual rows
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    
    #Turns time column into the right date time format
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    
    #Drops the na values from the value column
    melted = melted.dropna(subset=['value'])
    
    #Groups all rows by time and calculates the mean across each entry for that time
    return melted.groupby('time').aggregate({'value':'mean'}) 

In [ ]:
df = melt_data(data.drop(labels=['RegionID','SizeRank'], axis=1))

In [ ]:
df.head(20)

In [ ]:
df.shape

In [ ]:
rent = pd.read_csv('Data/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv')
rent.head()

In [ ]:
len(rent)

In [ ]:
Idaho = test2[test2['State']=='ID']
len(Idaho)

In [ ]:
plt.hist(np.clip(Idaho['2018-04'], 50000, 3000000), bins='auto');

In [ ]:
plt.hist(Idaho['2018-04'], bins='auto');

In [ ]:
plt.hist(Idaho['ROI'],bins='auto');

In [ ]:
Idaho['2018-04'].describe()